In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import numpy as np
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance

import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions
from src.analysis.config.plotting_parameters import plotting_parameters as plot_params
import src.analysis.utils.general_utils as utils # import utils functions one by one by name
from src.analysis.power.power_traces import make_multi_channel_evokeds_for_all_conditions_and_rois, plot_power_traces_for_all_rois, create_subtracted_evokeds_dict

['/hpc/home/etb28/miniconda3/envs/ieeg/lib/python311.zip', '/hpc/home/etb28/miniconda3/envs/ieeg/lib/python3.11', '/hpc/home/etb28/miniconda3/envs/ieeg/lib/python3.11/lib-dynload', '', '/hpc/home/etb28/miniconda3/envs/ieeg/lib/python3.11/site-packages']
Qt5Agg backend not available, using default backend
Qt5Agg backend not available, using default backend


get lab root for save dir

In [12]:
LAB_root = None
task='GlobalLocal'
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

LAB_root = '/cwork/etb28/'


# Get data layout
layout = get_data(task, root=LAB_root)
save_dir_root = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

print("setting path")

AssertionError: Could not find BIDS directory in /hpc/home/etb28/coganlab/Data/BIDS-1.1_GlobalLocal for task GlobalLocal

choose which subjects you wanna run (has to be a list, even if just one subject)

In [6]:
#taking out D0071, D0090, D0110, D0116 to see if it works
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0077', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0117', 'D0121']
# subjects = ['D0057']



### make or load subjects electrodes to rois dict

In [7]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = os.path.join(project_root, 'src', 'analysis', 'config')

subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                        filename='subjects_electrodestoROIs_dict.json')

print("subjects electrodes to rois dict")

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from /hpc/group/coganlab/etb28/GlobalLocal/src/analysis/config/subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


check number of elecs

In [8]:
# Calculate the total number of electrodes in the 'default_dict' across subjects
total_electrodes = utils.count_electrodes_across_subjects(subjects_electrodestoROIs_dict, subjects)
print('total elecs:', total_electrodes)

total elecs: 3283


load behavioral data (combinedData.csv) and map blockType to congruency and switch proportions in the behavioral data

In [9]:
combined_data = pd.read_csv(os.path.join(LAB_root, 'D_Data', 'GlobalLocal', 'combinedData.csv'))
combined_data[['switchProportion', 'congruencyProportion']] = combined_data.apply(utils.map_block_type, axis=1)

print("load behavioral")

### load epochs objects for chosen conditions for all subjects in a dictionary

In [10]:
task='GlobalLocal'
conditions = experiment_conditions.stimulus_iR_cS_err_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = [condition for condition in conditions.keys()]

print("load epochs")

In [ ]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_err_corr_conditions:
    conditions_save_name = 'stimulus_err_corr_conditions_sem_green' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_err_congruency_conditions:
    conditions_save_name = 'stimulus_err_congruency_conditions_sem' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_iR_cS_err_conditions:
    conditions_save_name = 'stimulus_err_iR_cS_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'


elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_err_corr_conditions:
    conditions_save_name = 'response_err_corr_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_err_congruency_conditions:
    conditions_save_name = 'response_err_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_iR_cS_err_conditions:
    conditions_save_name = 'response_err_iR_cS_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
#subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True, error_trials_only=False)
#one without selecting only correct trials
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=False)

Loading data for subject: D0057


BIDSValidationError: 'dataset_description.json' is missing from project root. Every valid BIDS dataset must have this file.
Example contents of 'dataset_description.json': 
{"Name": "Example dataset", "BIDSVersion": "1.0.2"}

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [ ]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103
Loaded significant channels for subject D0107A
Loaded significant channels for subject D0110
Loaded significant channels for subject D0116
Loaded significant channels for subject D0117
Loaded significant channels for subject D0121


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [ ]:

# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
# rois_dict = {
#    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
#}

rois_dict = {
     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
     'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
     'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
     'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
 }

rois = list(rois_dict.keys())
                                                
raw_all_electrodes_per_subject_roi, raw_sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

the new epochs root files that drop bad channels result in a channel mismatch with subjects electrodes to rois dict. Need to only use the electrodes that exist in the epochs data.

In [ ]:
# filter electrodes to only the ones that exist in the epochs objects. This mismatch can arise due to dropping channels when making the epochs objects, because the subjects_electrodestoROIs_dict is made based on all the electrodes, with no dropping.
sig_electrodes_per_subject_roi = utils.filter_electrode_lists_against_subjects_mne_objects(rois, raw_sig_electrodes_per_subject_roi, subjects_mne_objects)
all_electrodes_per_subject_roi = utils.filter_electrode_lists_against_subjects_mne_objects(rois, raw_all_electrodes_per_subject_roi, subjects_mne_objects)

dropped_sig_electrodes, _ = utils.find_difference_between_two_electrode_lists(raw_sig_electrodes_per_subject_roi, sig_electrodes_per_subject_roi)
print("\n--- Summary of Dropped Sig Electrodes ---")
total_dropped = 0
for roi, sub_dict in dropped_sig_electrodes.items():
    if not sub_dict: continue # Skip ROIs with no dropped electrodes
    print(f"ROI: {roi}")
    for sub, elec_list in sub_dict.items():
        if elec_list:
            print(f"  - Subject {sub}: Dropped {len(elec_list)} sig electrode(s)")
            total_dropped += len(elec_list)
print(f"Total sig electrodes dropped across all subjects/ROIs: {total_dropped}")
print("-------------------------------------\n")

In [ ]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 525


get number of sig and all electrodes per subject and across subjects

In [ ]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = all_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0057, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 17
Subject D0059, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 5
Subject D0063, ROI lpfc, Num of Sig Electrodes: 7, Num of All Electrodes: 36
Subject D0065, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 9
Subject D0069, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 1
Subject D0071, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 13
Subject D0077, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 0
Subject D0090, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 8
Subject D0094, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 28
Subject D0100, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 0
Subject D0102, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 5
Subject D0103, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 11
Subject D0107A, ROI lpfc, Num of Sig Electrodes: 1, Num of All Electrodes: 25
Subje

In [ ]:
# Example usage:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 44
Total number of lpfc electrodes across all subjects: 265


check sampling rates to make sure they're all the same

In [ ]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = utils.check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)

Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.
Subject D0107A has the expected sampling rate: 256.0 Hz.
Subject D0110 has the expected sampling rate: 256.0 Hz.
Subject D0116 has the expected sampling rate: 256.0 Hz.
Subject D0117 has the expected sampling rate: 256.0 Hz.
Subject D0121 has the expected sampling rate: 256.0 Hz.


TODO: stats next (maybe do time perm cluster with t-test as follow-up test for ANOVA significance, let's try using MNE functions for ANOVA and perm test)   
TODO: add in code for selecting electrodes of a specific ROI  
TODO: plotting average traces

now let's actually try plotting

In [ ]:
save_dir = os.path.join(save_dir_root, epochs_root_file)

evks_dict_sig_elecs = make_multi_channel_evokeds_for_all_conditions_and_rois(
    subjects_mne_objects, subjects, rois, condition_names, 
    sig_electrodes_per_subject_roi
) 

evks_dict_all_elecs = make_multi_channel_evokeds_for_all_conditions_and_rois(
    subjects_mne_objects, subjects, rois, condition_names, 
    all_electrodes_per_subject_roi
)

plot_power_traces_for_all_rois(
    evks_dict_all_elecs, rois, condition_names, conditions_save_name, plot_params,
    save_dir=save_dir,
    error_type='sem', figsize=(12, 8), 
    x_label='Time from Stimulus Onset (s)', 
    y_label='Power (z)',
    axis_font_size=35, tick_font_size=24, title_font_size=40, save_name_suffix='all_elecs'
)

plot_power_traces_for_all_rois(
    evks_dict_sig_elecs, rois, condition_names, conditions_save_name, plot_params,
    save_dir=save_dir,
    error_type='sem', figsize=(12, 8), 
    x_label='Time from Stimulus Onset (s)', 
    y_label='Power (z)',axis_font_size=35, tick_font_size=24,
    title_font_size=40, save_name_suffix='sig_elecs'
)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\power\power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RAI6', 'LFAM9', 'LFAM8'}. Applying running numbers for duplicates.
  info = mne.create_info(
c:\Users\jz421\Desktop\GlobalLocal\src\analysis\power\power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RAI6', 'LFAM9', 'LFAM8'}. Applying running numbers for duplicates.
  info = mne.create_info(


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\power\power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RIA14', 'RIA16', 'RIA15', 'LASF10', 'LFAM9', 'RIA5', 'LAI7', 'LAI8', 'RFAI3', 'RAI16', 'RAI12', 'LFO12', 'LFAM10', 'RAMF11', 'LFOP15', 'LFMI5', 'RAI14', 'RAMF12', 'RAMF13', 'RFO14', 'LFAM8', 'RAI11', 'RIA11', 'RAMF10', 'RAMF14', 'RASF15', 'RAI15', 'LFO16', 'LFO15', 'LFAM12', 'RFMM12', 'LFAI4', 'RFMM11', 'RIA6', 'RAI6', 'RMSF11', 'LFO13', 'RIA13', 'RFOA16', 'LFAM11', 'RASF16', 'RIA12', 'RMSF12'}. Applying running numbers for duplicates.
  info = mne.create_info(
c:\Users\jz421\Desktop\GlobalLocal\src\analysis\power\power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RIA14', 'RIA16', 'RIA15', 'LASF10', 'LFAM9', 'RIA5', 'LAI7', 'LAI8', 'RFAI3', 'RAI16', 'RAI12', 'LFO12', 'LFAM10', 'RAMF11', 'LFOP15', 'LFMI5', 'RAI14', 'RAMF12', 'RAMF13', 'RFO14', 'LFAM8', 'RAI11', 'RIA11', 'RAMF10', 'RAMF14', 'RASF15', 'RAI15'

ValueError: keyword fontsize is not recognized; valid keywords are ['size', 'width', 'color', 'tickdir', 'pad', 'labelsize', 'labelcolor', 'labelfontfamily', 'zorder', 'gridOn', 'tick1On', 'tick2On', 'label1On', 'label2On', 'length', 'direction', 'left', 'bottom', 'right', 'top', 'labelleft', 'labelbottom', 'labelright', 'labeltop', 'labelrotation', 'grid_agg_filter', 'grid_alpha', 'grid_animated', 'grid_antialiased', 'grid_clip_box', 'grid_clip_on', 'grid_clip_path', 'grid_color', 'grid_dash_capstyle', 'grid_dash_joinstyle', 'grid_dashes', 'grid_data', 'grid_drawstyle', 'grid_figure', 'grid_fillstyle', 'grid_gapcolor', 'grid_gid', 'grid_in_layout', 'grid_label', 'grid_linestyle', 'grid_linewidth', 'grid_marker', 'grid_markeredgecolor', 'grid_markeredgewidth', 'grid_markerfacecolor', 'grid_markerfacecoloralt', 'grid_markersize', 'grid_markevery', 'grid_mouseover', 'grid_path_effects', 'grid_picker', 'grid_pickradius', 'grid_rasterized', 'grid_sketch_params', 'grid_snap', 'grid_solid_ca

subtract conditions from each other for lwpc, lwps, and cross-construct comparisons

In [ ]:

if conditions == experiment_conditions.stimulus_lwpc_conditions:
    subtraction_pairs = [("Stimulus_i75", "Stimulus_c75"), ("Stimulus_i25", "Stimulus_c25")]
    subtraction_pairs_condition_names = ['-'.join(pair) for pair in subtraction_pairs]
    subtraction_pairs_conditions_save_name = 'stimulus_i75-c75_vs_i25-c25' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    subtracted_evks_dict_sig_elecs = create_subtracted_evokeds_dict(evks_dict_sig_elecs, subtraction_pairs, rois)
    subtracted_evks_dict_all_elecs = create_subtracted_evokeds_dict(evks_dict_all_elecs, subtraction_pairs, rois)

elif conditions == experiment_conditions.stimulus_lwps_conditions:
    subtraction_pairs = [("Stimulus_s75", "Stimulus_r75"), ("Stimulus_s25", "Stimulus_r25")]
    subtraction_pairs_condition_names = ['-'.join(pair) for pair in subtraction_pairs]
    subtraction_pairs_conditions_save_name = 'stimulus_s75-r75_vs_s25-r25' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    subtracted_evks_dict_sig_elecs = create_subtracted_evokeds_dict(evks_dict_sig_elecs, subtraction_pairs, rois)
    subtracted_evks_dict_all_elecs = create_subtracted_evokeds_dict(evks_dict_all_elecs, subtraction_pairs, rois)

elif conditions == experiment_conditions.stimulus_switch_type_by_congruency_proportion_conditions:
    raise NotImplementedError("this condition hasn't been made yet, also it would require reworking my create_subtracted_evokeds_dict to take in a tuple of lists of conditions for the subtraction")

elif conditions == experiment_conditions.stimulus_congruency_effect_by_switch_proportion_conditions:
    raise NotImplementedError("this condition hasn't been made yet, also it would require reworking my create_subtracted_evokeds_dict to take in a tuple of lists of conditions for the subtraction")
else:
    raise ValueError('tbh this should be a function, but for now, you cant plot subtracted evokeds for non subtraction conditions')    
    
plot_power_traces_for_all_rois(
    subtracted_evks_dict_all_elecs, rois, subtraction_pairs_condition_names, subtraction_pairs_conditions_save_name, plot_params,
    save_dir=save_dir,
    error_type='sem', figsize=(12, 8), 
    x_label='Time from Stimulus Onset (s)', 
    y_label='Power (z)',
    font_size=35, title_font_size=40, save_name_suffix='all_elecs'
)

plot_power_traces_for_all_rois(
    subtracted_evks_dict_sig_elecs, rois, subtraction_pairs_condition_names, subtraction_pairs_conditions_save_name, plot_params,
    save_dir=save_dir,
    error_type='sem', figsize=(12, 8), 
    x_label='Time from Stimulus Onset (s)', 
    y_label='Power (z)',
    font_size=35, title_font_size=40, save_name_suffix='sig_elecs'
)


AttributeError: module 'src.analysis.config.experiment_conditions' has no attribute 'stimulus_switch_type_by_congruency_proportion_conditions'

In [ ]:
# TODO: hm i should make a function that can take in stimulus_experiment_conditions and create evokeds based on chosen comparisons (i.e., i75,c75,i25,c25)